<a href="https://colab.research.google.com/github/sbj6364/piano-sequencer/blob/main/09_synthesizer.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 9번 : 피아노 시퀀서

이제 우리는 12개의 반음을 원하는 길이 만큼 변화시키고 이어붙이는 방법에 대해 알게 되었습니다.

지금까지의 내용을 종합해서, 음정 및 duration 입력 sequence를 받아 해당 입력을 반영한 음원을 생성해 저장하는 코드를 작성해주세요.

#### 베이스라인
~~~python
def synthesizer(pitch, duration):
    ''' code here '''
    librosa.output.write_wav('output.wav', wave, sr)
~~~

In [75]:
import matplotlib.pyplot as plt
import librosa
import librosa.display
import numpy as np 

In [ ]:
import librosa
do, sr_do = librosa.load('do.mp3', sr=None)
re, sr_re = librosa.load('re.mp3', sr=None)
mi, sr_mi = librosa.load('mi.mp3', sr=None)
fa, sr_fa = librosa.load('fa.mp3', sr=None)
sol, sr_sol = librosa.load('sol.mp3', sr=None)
ra, sr_ra = librosa.load('ra.mp3', sr=None)
si, sr_si = librosa.load('si.mp3', sr=None)

In [ ]:
# 방법 1
piano = {'do':do, 're':re, 'mi':mi, 'fa':fa, 'sol':sol, 'ra':ra, 'si':si}
piano.values()

In [ ]:
def LinearSpectrogram(wav):
		stft = librosa.stft(wav)
		stft = np.abs(stft)
		return stft

plt.figure(figsize=(16,9))
for index, item in enumerate(piano.keys()):
    wave = piano[item]
    # print(wave)
    stft = LinearSpectrogram(wave)
    plt.subplot(1,7,index+1)
    librosa.display.specshow(np.log(stft+1e-5))
    plt.title(item)

plt.tight_layout()
plt.savefig('LinearSpec.png')

In [ ]:
def ChromaSpectrogram(wav):
		chroma = librosa.feature.chroma_stft(wav)
		chroma = np.abs(chroma)
		return chroma

plt.figure(figsize=(16,9))
for index, item in enumerate(piano.keys()):
    wave = piano[item]
    c_stft = ChromaSpectrogram(wave)
    plt.subplot(1,7,index+1)
    librosa.display.specshow(c_stft)
    plt.title(item)

plt.tight_layout()
plt.savefig('Chroma.png')

In [ ]:
def convert(a):
  if a == 'C':
    a = 'do'
  elif a == 'D':
    a = 're'
  elif a == 'E':
    a = 'mi'
  elif a == 'F':
    a = 'fa'
  elif a == 'G':
    a = 'sol'
  elif a == 'A':
    a = 'ra'
  elif a == 'B':
    a = 'si'
  return a


def sum_and_plot(a,b):
  plt.figure(figsize=(16,9))
  a_s = convert(a)
  b_s = convert(b)

  wave_a = piano[a_s]
  wave_b = piano[b_s]
  wave_ab = piano[a_s] + piano[b_s]

  stft_a = LinearSpectrogram(wave_a)
  plt.subplot(2,3,1)
  librosa.display.specshow(np.log(stft_a+1e-5))
  plt.title('STFT'+a)

  stft_b = LinearSpectrogram(wave_b)
  plt.subplot(2,3,2)
  librosa.display.specshow(np.log(stft_b+1e-5))
  plt.title('STFT'+b)

  stft_ab = LinearSpectrogram(wave_ab)
  plt.subplot(2,3,3)
  librosa.display.specshow(np.log(stft_ab+1e-5))
  plt.title('STFT'+a+'+'+b)

  c_stft_a = ChromaSpectrogram(wave_a)
  plt.subplot(2,3,4)
  librosa.display.specshow(c_stft_a)
  plt.title('Chroma'+a)

  c_stft_b = ChromaSpectrogram(wave_b)
  plt.subplot(2,3,5)
  librosa.display.specshow(c_stft_b)
  plt.title('Chroma'+b)

  c_stft_ab = ChromaSpectrogram(wave_ab)
  plt.subplot(2,3,6)
  librosa.display.specshow(c_stft_ab)
  plt.title('Chroma'+a+'+'+b)

  plt.tight_layout()
  plt.savefig('sum_and_plot.png')
 
a = input()
b = input()

sum_and_plot(a,b)

In [ ]:
def concat_piano(arr):
  result = []
  for sound in arr:
    a = convert(sound)
    result = np.append(result, piano[a])
  return result

s = input()
s_l = list(s)

wave = concat_piano(s_l)

plt.figure(figsize=(16,9))

stft = LinearSpectrogram(wave)
plt.subplot(2,1,1)
librosa.display.specshow(np.log(stft+1e-5))
plt.title('STFT')

c_stft = ChromaSpectrogram(wave)
plt.subplot(2,1,2)
librosa.display.specshow(c_stft)
plt.title('Chroma')

plt.tight_layout()
plt.savefig('concat.png')

In [ ]:
def pitch_shift(audio, sr, semitone):
		shifted = librosa.effects.pitch_shift(audio, sr, semitone)
		return shifted

plt.close()
plt.figure(figsize=(16,9))

plt.subplot(1,3,1)
librosa.display.specshow(ChromaSpectrogram(piano['do']))
plt.title('C')

plt.subplot(1,3,2)
librosa.display.specshow(ChromaSpectrogram(pitch_shift(piano['do'], sr, 1)))
plt.title('C->C#')

plt.subplot(1,3,3)
librosa.display.specshow(ChromaSpectrogram(pitch_shift(piano['do'], sr, 2)))
plt.title('C->D')

plt.tight_layout()
plt.savefig('pitch_shift.png')

In [ ]:
def concat_piano(pitch, duration):
  concat = np.zeros((0))
  for index, item in enumerate(pitch):
    # print(index, item, convert(item))
    a = convert(item)
    tmp = librosa.effects.time_stretch(piano[a], 1/duration[index])
    concat = np.append(concat, tmp)

  plt.figure(figsize=(16,9))
  plt.subplot(2,1,1)
  librosa.display.specshow(np.log(LinearSpectrogram(concat)+1e-5))
  plt.title('STFT')

  plt.subplot(2,1,2)
  librosa.display.specshow(ChromaSpectrogram(concat))
  plt.title('chroma')

  plt.tight_layout()
  plt.savefig('concat_duration.png')

concat_piano('EDCDEEE', [2,0.5,1,1,1,1,3])

In [179]:
import soundfile as sf

In [188]:
def synthesizer(pitch, duration):
  wave = np.zeros((0))
  for index, item in enumerate(pitch):
    a = convert(item)
    tmp = librosa.effects.time_stretch(piano[a], 1/duration[index])
    wave = np.append(wave, tmp)
    
    sf.write('output.wav', wave, sr)

pitch = list(input())
duration = input().split(',')

# pitch = list('GFEG')
# duration = [0.5, 0.2, 0.3, 1]

duration = list(map(float, duration))

synthesizer(pitch, duration)

In [189]:
ipd.Audio('output.wav')